## Machine Learning 프로젝트 수행을 위한 코드 구조화 

- ML project를 위해서 사용하는 템플릿 코드를 만듭니다.

1. **필요한 라이브러리와 데이터를 불러옵니다.**


2. **EDA를 수행합니다.** 이 때 EDA의 목적은 풀어야하는 문제를 위해서 수행됩니다.


3. **전처리를 수행합니다.** 이 때 중요한건 **feature engineering**을 어떻게 하느냐 입니다.


4. **데이터 분할을 합니다.** 이 때 train data와 test data 간의 분포 차이가 없는지 확인합니다.


5. **학습을 진행합니다.** 어떤 모델을 사용하여 학습할지 정합니다. 성능이 잘 나오는 GBM을 추천합니다.


6. **hyper-parameter tuning을 수행합니다.** 원하는 목표 성능이 나올 때 까지 진행합니다. 검증 단계를 통해 지속적으로 **overfitting이 되지 않게 주의**하세요.


7. **최종 테스트를 진행합니다.** 데이터 분석 대회 포맷에 맞는 submission 파일을 만들어서 성능을 확인해보세요.

## 1. 라이브러리, 데이터 불러오기

In [ ]:
# 설치에 필요한 라이브러리들이 있다면 모두 적어둡니다. anaconda에 기본적으로 설치되지 않은 라이브러리들을 적어두세요.
!pip install lightgbm optuna

In [ ]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 모델들, 성능 평가
# (저는 일반적으로 정형데이터로 머신러닝 분석할 때는 이 2개 모델은 그냥 돌려봅니다. 특히 RF가 테스트하기 좋습니다.)
from sklearn.ensemble import RandomForestRegressor
from lightgbm.sklearn import LGBMRegressor

# 상관관계 분석, VIF : 다중공선성 제거
from statsmodels.stats.outliers_influence import variance_inflation_factor

# KFold(CV), partial : optuna를 사용하기 위함
from sklearn.model_selection import KFold
from functools import partial

# hyper-parameter tuning을 위한 라이브러리, optuna
import optuna

In [ ]:
# 데이터를 불러옵니다.
train = pd.read_csv('../input/spaceship-titanic/train.csv')
test = pd.read_csv('../input/spaceship-titanic/test.csv')

## 2. EDA

- 데이터에서 찾아야 하는 기초적인 내용들을 확인합니다.


- class imbalance, target distribution, outlier, correlation을 확인합니다.

In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
test.info()

In [ ]:
for col in train.columns:
    print('----------------------------\n')
    print(train[col].value_counts())
    print('----------------------------\n')

In [ ]:
for col in test.columns:
    print('----------------------------\n')
    print(test[col].value_counts())
    print('----------------------------\n')

In [ ]:
train.Cabin

In [ ]:
train.Cabin.str[0].value_counts()

In [ ]:
train['Cabin_deck'] = train.Cabin.str[0]
test['Cabin_deck'] = test.Cabin.str[0]
print(train['Cabin_deck'].value_counts())
pd.pivot_table(data=train, index = 'Cabin_deck', values='Transported', aggfunc= ['mean','sum'])

In [ ]:
train['Cabin_num'] = train.Cabin.str[2:-2]
test['Cabin_num'] = test.Cabin.str[2:-2]
print(train['Cabin_num'].value_counts())
pd.pivot_table(data=train, index = 'Cabin_num', values='Transported', aggfunc= ['mean','sum'])

In [ ]:
train['Cabin_side'] = train.Cabin.str[-1]
test['Cabin_side'] = test.Cabin.str[-1]
print(train['Cabin_side'].value_counts())
pd.pivot_table(data=train, index = 'Cabin_side', values='Transported', aggfunc= ['mean','sum'])
# train.Cabin.str[-1].value_counts()

In [ ]:
train = train.drop(columns='Cabin')
test = test.drop(columns='Cabin')

In [ ]:
#  테스트 결과 효용성이 없다고 판단
# RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.
# train['Bill'] = train.RoomService + train.FoodCourt + train.ShoppingMall + train.Spa + train.VRDeck
# test['Bill'] = test.RoomService + test.FoodCourt + test.ShoppingMall + test.Spa + test.VRDeck

In [ ]:
# train = train.drop(columns=['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'])
# test = test.drop(columns=['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'])

In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
# 목적지에 따라 transported 의 비중이 다르진 않은지..
pd.pivot_table(data=train, index='Destination', values='Transported', aggfunc= 'mean')

In [ ]:
plt.figure(figsize=(12,6))
sns.histplot(data=train, x="Age")
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
plt.ylim(0,100)
sns.histplot(data=train, x="RoomService")
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
plt.ylim(0,100)
sns.histplot(data=train, x="FoodCourt")
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
plt.xlim(0,1000)
plt.ylim(0,100)
sns.histplot(data=train, x="ShoppingMall")
plt.show()

In [ ]:
corr = train.drop(columns=["PassengerId", 'Name']).corr()
plt.figure(figsize=(20,20))
sns.heatmap(corr, annot=True)

correlation 분석 결과 상관계수가 높은 feature 는 나타나지 않음

### 3. 전처리

#### 결측치 처리

In [ ]:
print(train.columns)
print(test.columns)
# print(train.columns[train.isnull().any()])
# has_null = train.isnull().any()

In [ ]:
y = train.Transported
train = train.drop(columns='Transported')

In [ ]:
print(train.info())
print('-----------------------------')
print(test.info())

In [ ]:
for col in train.columns:
    if train[col].dtype == 'object':
        train[col] = train[col].fillna(train[col].mode()[0])
        test[col] = test[col].fillna(train[col].mode()[0])
    elif train[col].dtype == 'float64':
        train[col] = train[col].fillna(train[col].median())
        test[col] = test[col].fillna(train[col].median())

In [ ]:
train.isnull().any()

In [ ]:
test.isnull().any()

In [ ]:
drop_cols = ['PassengerId','Name','Cabin_num']
train = train.drop(columns=drop_cols)
test = test.drop(columns=drop_cols)
    
# train = train.drop(columns=['PassengerId','Name'])
# test = test.drop(columns=['PassengerId','Name'])

# train = train.drop(columns=['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'])
# test = test.drop(columns=['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'])

In [ ]:
for col in train.columns:
    if train[col].dtype == 'object':
        print('categorical encoding for ['+col+']\n')
        train[col] = pd.factorize(train[col])[0]
        test[col] = pd.factorize(test[col])[0]
        

In [ ]:
corr = train.corr()
plt.figure(figsize=(20,20))
sns.heatmap(corr, annot=True)

In [ ]:
# train.HomePlanet = train.HomePlanet.fillna(train.HomePlanet.mode()[0])

In [ ]:
# train.CryoSleep = train.CryoSleep.fillna(train.CryoSleep.mode()[0])

In [ ]:
# train.Destination = train.Destination.fillna(train.Destination.mode()[0])

In [ ]:
# train.Age = train.Age.fillna(train.Age.mean())

In [ ]:
# 결측치가 있는 column


In [ ]:
# 각 feature들이 discretes한지 continuous한지 확인합니다.


In [ ]:
# 중복정보가 있는 column 제거하기 위해 상관계수를 확인해봅니다.
# correlated_features = 

### 4. 학습 데이터 분할

In [ ]:
# 첫번째 테스트용으로 사용하고, 실제 학습시에는 K-Fold CV를 사용합니다.
from sklearn.model_selection import train_test_split

X = train

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

### 5. 학습 및 평가

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=5 , random_state=42)
clf.fit(X_train, y_train) # training

from sklearn.metrics import accuracy_score

pred = clf.predict(X_train)
pred2 = clf.predict(X_test)

print('Training Acc : %.4f' % accuracy_score(y_train, pred))
print('Validation Acc : %.4f' % accuracy_score(y_test, pred2))

### 6. Hyper-parameter Tuning

> GridSearchCV

In [ ]:
from sklearn.model_selection import cross_val_score

def optimizer(trial):
    max_depth = trial.suggest_int('max_depth', 1, 10)
    max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 2, 1000)
    n_estimators =  trial.suggest_int('n_estimators', 100, 500)
   
    model = RandomForestClassifier(max_depth = max_depth, 
                                   max_leaf_nodes = max_leaf_nodes,
                                   n_estimators = n_estimators,
                                   n_jobs=2,
                                   random_state=25)

    
    model.fit(X_train, y_train)    
    score = cross_val_score(model, X_train, y_train, cv=5, scoring="f1")
    f1_mean = score.mean()

    return f1_mean
    
#Execute optuna and set hyperparameters
study = optuna.create_study(direction='maximize')
study.optimize(optimizer, n_trials = 100)



In [ ]:
study.trials_dataframe()

In [ ]:
print("Best Score: %.4f" % study.best_value) # best score 출력
print("Best params: ", study.best_trial.params) # best score일 때의 하이퍼파라미터들

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
# hyper-parameter들의 중요도
optuna.visualization.plot_param_importances(study)

### 7. 테스트 및 제출 파일 생성

In [ ]:
print(study.best_params['max_depth'])
print(study.best_params['max_leaf_nodes'])
print(study.best_params['n_estimators'])

In [ ]:
#Create an instance with tuned hyperparameters
model = RandomForestClassifier(max_depth = study.best_params['max_depth'], 
                                      max_leaf_nodes = study.best_params['max_leaf_nodes'],
                                      n_estimators = study.best_params['n_estimators'],
                                      n_jobs=2,
                                      random_state=42)

In [ ]:
model.fit(train, y)
preds = model.predict(test)

In [ ]:
submission = pd.read_csv('../input/spaceship-titanic/sample_submission.csv')
submission['Transported'] = preds
submission

In [ ]:
submission.reset_index(drop=True).to_csv("submission.csv", index=False)